In [4]:
import sys
sys.path.append('E:/zlab/')
from loader import Loader, SemiXModel, XModel
# ------------------------------------
import time
import numpy as np
from sklearn.model_selection import train_test_split

import tables as tb
import mxnet as mx
from mxnet import nd, autograd, gluon
import gluonbook as gb
from mxnet import gluon, init, nd
from mxnet.gluon import data as gdata, loss as gloss, model_zoo

h5 = tb.open_file('E:/xdata/X.h5')

data = h5.root.cifar10

X = data.trainX[:]
y = data.trainY[:]
X_train, X_unlabel, y_train, y__unlabel = train_test_split(
    X, y, test_size=0.98, random_state=42)

batch_size = 32
trainset = Loader(X_train, y_train, batch_size, shuffle=True)
testset = Loader(data.testX, data.testY, batch_size, shuffle=False)
unlabelset = Loader(X_unlabel, y__unlabel, 128, shuffle=True)

# ------------
pretrained_net = model_zoo.vision.resnet50_v2(pretrained=True)
finetune_net = model_zoo.vision.resnet50_v2(classes=10)
finetune_net.features = pretrained_net.features
finetune_net.output.initialize(init.Xavier())

In [3]:
learning_rate = 0.05
epochs = 100

SM = SemiXModel(trainset, testset, unlabelset, finetune_net)

#net = SM.train(learning_rate, batch_size, epochs)

In [5]:
XM = XModel(trainset, testset, finetune_net)

In [11]:
for features, labels in iter(XM.testset):
    ys = nd.array(labels)
    Xs = nd.stack(*[XM.aug.test(nd.array(x)) for x in features])
    Xs = Xs.as_in_context(XM.ctx)
    y_hats = XM.net(Xs)
    
    Y_ = nd.concat(*[y_hats])
    break

In [12]:
y_hats


[[-2.49721527e-01  1.52786934e+00 -3.43045443e-01 -5.10813355e-01
   7.40163267e-01  3.63533288e-01  8.51983190e-01 -1.59381032e+00
   1.67389452e+00 -2.84208745e-01]
 [-9.50222313e-02  2.61418581e+00 -1.14246893e+00 -2.93449372e-01
   1.07296491e+00  5.21471560e-01  9.71140563e-01 -1.02084458e-01
   4.91728902e-01 -6.24709666e-01]
 [-9.74178612e-01  2.09942675e+00 -3.18102151e-01 -6.88672125e-01
   6.63622737e-01  6.29193068e-01  1.49738669e+00  5.21191001e-01
   1.71566463e+00 -4.54443097e-01]
 [-1.23794365e+00  2.10908914e+00 -1.99633884e+00 -2.93017316e+00
   1.74743736e+00  2.17181206e-01  2.10669208e+00 -1.76511943e-01
   6.84043646e-01 -1.88615739e+00]
 [ 3.01124036e-01  2.89109468e+00 -2.27055001e+00 -1.96049571e+00
  -6.17368817e-02  1.84199065e-01  8.59018564e-01 -1.21635079e+00
   4.29620624e-01  4.78704512e-01]
 [ 7.66122937e-01  7.71434128e-01  1.10347599e-01 -1.24498200e+00
  -1.00844061e+00  8.97678077e-01  1.92166162e+00 -2.73866057e-01
   1.44086552e+00  1.38869494e-0